## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**

  * 関数名: `project_employees`
  * 引数名: `project`, `employee`
  * 返却列: `["project_id", "average_years"]`
  * 列順: 上記順序
* I/O 禁止（ファイル / 標準出力）、`print` / `sort_values` は使用しない

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  各プロジェクトについて、そのプロジェクトにアサインされている従業員の
  **平均経験年数 (`experience_years`) を小数第 2 位に丸めて** 求める。

* 入力 DF: `{{INPUT_DATAFRAMES}}`

  * `project: pd.DataFrame`

    | column      | dtype |
    | ----------- | ----- |
    | project_id  | int   |
    | employee_id | int   |

    各行は「従業員 `employee_id` がプロジェクト `project_id` に所属している」ことを表す。

  * `employee: pd.DataFrame`

    | column           | dtype  |
    | ---------------- | ------ |
    | employee_id      | int    |
    | name             | object |
    | experience_years | int    |

    各行は従業員 1 名の情報。`experience_years` は NULL なし。

* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`

  * 戻り値: `pd.DataFrame`

  * 列と意味:

    * `project_id`: プロジェクト ID
    * `average_years`: そのプロジェクトに所属する従業員の `experience_years` の平均値（小数第 2 位で丸め）

  * 各 `project_id` につき 1 行

  * 並び順は任意（`sort_values` 禁止のためソートしない）

---

## 2) 実装（指定シグネチャ厳守）

> 列を最小化しつつ `merge` → `groupby.mean` → `round` の順に処理します。
> 今回はグループ内順位や条件抽出は不要なので、シンプルな集約だけで OK です。

```python
import pandas as pd

def project_employees(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    """
    各プロジェクトごとの平均経験年数を計算する。

    Args:
        project (pd.DataFrame): 列 ['project_id', 'employee_id']
        employee (pd.DataFrame): 列 ['employee_id', 'name', 'experience_years']

    Returns:
        pd.DataFrame: 列名と順序は ['project_id', 'average_years']
    """
    # 1) 列最小化: employee 側は平均に必要な列だけに絞る
    emp_exp = employee[["employee_id", "experience_years"]]

    # 2) JOIN: project に experience_years を紐づける
    merged = project.merge(emp_exp, on="employee_id", how="left")

    # 3) プロジェクトごとに平均値を計算
    out = (
        merged
        .groupby("project_id", as_index=False)["experience_years"]
        .mean()
    )

    # 4) 列名を仕様どおりにリネームし、小数第 2 位に丸める
    out = out.rename(columns={"experience_years": "average_years"})
    out["average_years"] = out["average_years"].round(2)

    return out

Analyze Complexity
Runtime 283 ms
Beats 62.61%
Memory 69.14 MB
Beats 18.00%

```

---

## 3) アルゴリズム説明

### 使用 API

* `DataFrame[...]`
  → 列のサブセットを取り、**列最小化**（不要列を運ばないことでメモリ削減）。
* `DataFrame.merge`
  → `project` と `employee` を `employee_id` で結合し、従業員の経験年数をプロジェクトに紐付け。
* `DataFrame.groupby` + `GroupBy.mean`
  → `project_id` ごとに `experience_years` の平均を計算。
* `DataFrame.rename`
  → 出力列名を問題仕様どおり `average_years` に。
* `Series.round`
  → 平均値を小数第 2 位に丸める。

### NULL / 重複 / 型の扱い

* `experience_years` は問題文より **NULL なし** 前提なので、`mean()` で NULL ケアは不要。
* `project` 側は `(project_id, employee_id)` が主キーなので、
  「同じプロジェクトに同じ従業員が複数行いる」ことはなく、重複による二重カウントも発生しない。
* `mean()` の戻り値は `float64`。
  `round(2)` で `2.0` → 画面では `2.00` 相当の精度を保持できる（pandas の表示設定次第）。

---

## 4) 計算量（概算）

`N = len(project)`, `M = len(employee)` とします。

* 列最小化: `employee[["employee_id", "experience_years"]]`
  → **O(M)**
* `merge`（ハッシュ結合想定）: `project.merge(emp_exp, on="employee_id")`
  → **O(N + M)** 近辺
* `groupby("project_id").mean()`
  → **O(N)**（ハッシュベースのグループ集計）

したがって、全体はおおよそ **O(N + M)** 時間、メモリは
結合後の一時 DataFrame（行数 ≒ N、列数は 3 列）相当となります。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[project<br/>project_id, employee_id]
  B[employee<br/>employee_id, experience_years, name]
  C[列最小化<br/>employee → employee_id, experience_years のみ]
  D[merge<br/>on employee_id]
  E["groupby project_id<br/>mean(experience_years)"]
  F["列名変更 + round(2)<br/>average_years"]
  G[出力<br/>project_id, average_years]

  B --> C
  A --> D
  C --> D
  D --> E
  E --> F
  F --> G
```

この形なら、LeetCode の pandas 版「Project Employees I」にそのままコピペ投入できるはずです。

結論から言うと、

* **計算量レベルではほぼ限界まで来ていて、大きな高速化は期待しにくい**
* ただし、**merge → map に変えることで、メモリとわずかな実行時間改善の余地はある**
* 特にメモリ使用量（Beats 18%）を少しでも良くしたいなら、「結合の仕方」を軽くするのがポイント

という感じです。

---

## 1) いまのアプローチの整理

あなたの現行コード（要約）は：

```python
emp_exp = employee[["employee_id", "experience_years"]]

merged = project.merge(emp_exp, on="employee_id", how="left")

out = (
    merged
    .groupby("project_id", as_index=False)["experience_years"]
    .mean()
)

out = out.rename(columns={"experience_years": "average_years"})
out["average_years"] = out["average_years"].round(2)
```

やっていることは完全に正しくて、アルゴリズム的にも

* 結合: O(N + M)
* groupby: O(N)

で、ここから**オーダーを変える改善はできません**。

LeetCode の 283ms / Beats 62% という数字も、
環境ノイズを含めて「十分良い」側です。

---

## 2) 改善ポイント：`merge` → `map` で軽量化

`project` 側にはすでに `employee_id` が入っているので、

> わざわざ `merge` で行を膨らませるのではなく、
> **`employee_id → experience_years` のマッピングを作って `map` する**

方が、メモリ的には少し有利になり得ます。

### 修正版コード（`map` ベース）

```python
import pandas as pd

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    """
    各プロジェクトごとの平均経験年数を計算する。

    Args:
        project (pd.DataFrame): 列 ['project_id', 'employee_id']
        employee (pd.DataFrame): 列 ['employee_id', 'name', 'experience_years']

    Returns:
        pd.DataFrame: 列名と順序は ['project_id', 'average_years']
    """
    # 1) employee_id -> experience_years のマップを作る（列最小化 + index 化）
    emp_exp = employee.set_index("employee_id")["experience_years"]

    # 2) project に対応する経験年数列を付与（merge の代わりに map）
    #    列最小化: project 側も必要な列だけ使う
    proj = project[["project_id", "employee_id"]].copy()
    proj["experience_years"] = proj["employee_id"].map(emp_exp)

    # 3) 集約に不要な employee_id を削る（メモリ削減の一手）
    proj = proj[["project_id", "experience_years"]]

    # 4) プロジェクトごとに平均値を計算
    out = (
        proj
        .groupby("project_id", as_index=False)["experience_years"]
        .mean()
    )

    # 5) 列名変更 + 小数第2位に丸める
    out = out.rename(columns={"experience_years": "average_years"})
    out["average_years"] = out["average_years"].round(2)

    return out

Analyze Complexity
Runtime 283 ms
Beats 62.61%
Memory 68.14 MB
Beats 98.61%

```

### この書き方の狙い

* `merge` は「左・右両方の列をフルで抱えた新しい DataFrame」を作るので、そのぶんメモリを食いやすい
* `map` は「Series の値を 1 列だけ投げ込む」だけなので、結合の中では比較的軽い
* `employee` 側は `set_index` して `Series` 化していて、
  **キー→値** のマップとして最小限の形にしている

大幅なメモリ削減にはならないかもしれませんが、

* 一時的に作るオブジェクトがやや小さくなる
* 実行時間も微妙に改善する可能性はある

ので、LeetCode のメモリ percentile を数ポイント押し上げられる可能性はあります。

---

## 3) どこまで改善を狙うべきか？

正直なところ、今回の問題は

* 入力サイズもそれほど極端ではない
* アルゴリズムも「JOIN + GROUP BY」で頭打ち

なので、**すでに「クエリとしては十分に最適な部類」** です。

283ms / Beats 62% という数字は、少しコードをいじったくらいで
劇的に変わるものではないので、

* 可読性
* 素直さ（変なトリックを使わない）
* バグの入りにくさ

を優先しつつ、上の `map` 版のような「軽い改善」を入れておけばかなり良いラインだと思います。

---

もしこの後、

* 別の LeetCode pandas 問題
* もっと重いグループ演算（複数条件 / 上位 k / window関数っぽい処理）

などが出てきたら、そこは **`groupby.transform` / `rank` / `merge` 戦略**を総合的に組み立てる練習ネタにできます。
